# Initial Insights
Notebook to derive some initial insights from the [@mediapermata](https://www.instagram.com/mediapermata/) Instagram page (Brunei's premier newspaper) with a particular focus on the comments.

In [1]:
# Load in necessary libraries
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import malaya
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import utils

In [2]:
user_name = utils.username
pass_word = utils.password

path = r'/Users/chong02/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get('https://instagram.com/')

# Logging in
time.sleep(10)
username=driver.find_element(By.NAME, 'username')
password=driver.find_element(By.NAME, 'password')
username.send_keys(user_name)
password.send_keys(pass_word)
password.send_keys(Keys.RETURN)

# Bypass save log in info
time.sleep(10)
not_now = driver.find_element(By.XPATH, "//button[contains(text(), 'Not Now')]").click()

# Bypass notifs
time.sleep(60)
not_now_again = driver.find_element(By.XPATH, "//button[contains(text(), 'Not Now')]").click()